## Preparation

In [1]:
import torch.nn as nn
import sys
from matplotlib import pyplot as plt
%matplotlib inline
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# For drive
path = "/content/drive/My Drive/project/"

sys.path.append(path + "URP")
sys.path.append(path + "URP/learning")
sys.path.append(path + "URP/data")
sys.path.append(path + "URP/models")
sys.path.append(path + "URP/forgetting")
sys.path.append(path + "URP/evaluation")

In [4]:
'''
# For PC
path = ".."
sys.path.append(path)
sys.path.append(path + "/learning")
sys.path.append(path + "/data")
sys.path.append(path + "/models")
sys.path.append(path + "/forgetting")
'''

'\n# For PC\npath = ".."\nsys.path.append(path)\nsys.path.append(path + "/learning")\nsys.path.append(path + "/data")\nsys.path.append(path + "/models")\nsys.path.append(path + "/forgetting")\n'

In [4]:
from data_utils import *#show_random, AddNoise, remove_random, remove_class, combine_datasets
from data import get_dataset, dummy_clusters, dummy_half_doughnuts
from models_utils import *
from eval import *
import models
from learn import *
from forget import *
from utils import set_seed

In [5]:
set_seed()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

SEED SET TO:  13


device(type='cuda', index=0)

##Models and Datasets

In [6]:
cifar10_train, cifar10_val, cifar10_test = get_dataset("cifar10", batch_size=32, cuda=0,
                  train_size=10000, val_size=2000, test_size=1000,
                  test_batch_size=32)
print('-'*20)

cnn = {"filters_percentage": 1, "n_channels":3,"num_classes": 50, "dropout": True, "batch_norm": True}
cnn_model = get_model("cnn", **cnn)
print(cnn_model.__class__.__name__)

Files already downloaded and verified
Dataset sizes: 	 train: 10000 	 val: 2000 	 test: 1000
Batch size: 	 32
--------------------
CNN


In [8]:
mnist_train, mnist_val, mnist_test = get_dataset("mnist", batch_size=32, cuda=0,
                  train_size=10000, val_size=2000, test_size=1000,
                  test_batch_size=32)
print('-'*20)

dnn = {"input_size": 1024, "hidden_size":64, "num_classes":10,"num_layer": 10, "activation": nn.ReLU()}
dnn_model = get_model("mlp", **dnn)
print(dnn_model.__class__.__name__)
print('-'*20)

checkpoint = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{dnn_model.__class__.__name__}_{50}.pth.tar")
dnn_model.load_state_dict(checkpoint['model'])
_ = test(model=dnn_model, loss='ce', lossfn=None, optimizer='sgd', device=device, dataset='mnist', test_loader=mnist_test, at_epoch=None)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/train-images-idx3-ubyte.gz to /root/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/data/MNIST/raw

Dataset sizes: 	 train: 10000 	 val: 2000 	 test: 1000
Batch size: 	 32
--------------------
MLP
--------------------
SEED SET TO:  13
Optimizer: 	  sgd

Loss function:
CrossEntropyLoss()
TESTING
Loss:  0.027804579585790634
Error:  0.0
FINISHED TESTING


In [11]:
cifar100_train, cifar100_val, cifar100_test = get_dataset("cifar100", batch_size=32, cuda=0,
                  train_size=20000, val_size=2000, test_size=1000,
                  test_batch_size=10)
print('-'*20)
#---------
resnet18 = {"filters_percentage": 0.4, "n_channels":3,"num_classes": 100, "block": ResBlock, "num_blocks": [2,2,2,2]}
resnet18_origin = get_model("resnet18", **resnet18)
print(resnet18.__class__.__name__)
print('-'*20)
'''
#---------
checkpoint_origin = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{resnet18_model.__class__.__name__}_{200}.pth.tar")
checkpoint_forget = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{resnet18_model.__class__.__name__}_{200}.pth.tar")
checkpoint_retrain = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{resnet18_model.__class__.__name__}_{200}.pth.tar")
#---------
resnet18_origin.load_state_dict(checkpoint_origin['model'])
resnet18_forget = copy.deepcopy(resnet18_origin)
resnet18_forget.load_state_dict(checkpoint_forget['model'])
resnet18_retrain = copy.deepcopy(resnet18_origin)
resnet18_retrain.load_state_dict(checkpoint_retrain['model'])
'''

Files already downloaded and verified
Dataset sizes: 	 train: 20000 	 val: 2000 	 test: 1000
Batch size: 	 32
--------------------
dict
--------------------


'\n#---------\ncheckpoint_origin = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{resnet18_model.__class__.__name__}_{200}.pth.tar")\ncheckpoint_forget = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{resnet18_model.__class__.__name__}_{200}.pth.tar")\ncheckpoint_retrain = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{resnet18_model.__class__.__name__}_{200}.pth.tar")\n#---------\nresnet18_origin.load_state_dict(checkpoint_origin[\'model\'])\nresnet18_forget = copy.deepcopy(resnet18_origin)\nresnet18_forget.load_state_dict(checkpoint_forget[\'model\'])\nresnet18_retrain = copy.deepcopy(resnet18_origin)\nresnet18_retrain.load_state_dict(checkpoint_retrain[\'model\'])\n'

In [7]:
checkpoint = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{cnn_model.__class__.__name__}_{70}.pth.tar")
cnn_model.load_state_dict(checkpoint['model'])
test(model=cnn_model, loss='ce', lossfn=None, optimizer='adam', device=device, dataset='cifar', test_loader=cifar10_test, at_epoch=None)
print("-"*20)
cnn_forget = copy.deepcopy(cnn_model)
checkpoint = torch.load(f"/content/drive/My Drive/project/URP/checkpoints/{cnn_model.__class__.__name__}_forget_{20}.pth.tar")
cnn_forget.load_state_dict(checkpoint['model'])
test(model=cnn_forget, loss='ce', lossfn=None, optimizer='adam', device=device, dataset='cifar', test_loader=cifar10_test, at_epoch=None)

SEED SET TO:  13
Optimizer: 	  adam

Loss function:
CrossEntropyLoss()
TESTING
Loss:  0.9684426188468933
Error:  0.125
FINISHED TESTING
--------------------
SEED SET TO:  13
Optimizer: 	  adam

Loss function:
CrossEntropyLoss()
TESTING
Loss:  1.8938524723052979
Error:  0.375
FINISHED TESTING


## Evaluation

In [8]:
distance(cnn_model, cnn_forget)

Distance: 80.50011464838822
Normalized Distance: 0.5218581232571371


0.5218581232571371

In [9]:
KL(cnn_model, cnn_forget)

AttributeError: ignored

In [10]:
entropy(cnn_model)

TypeError: ignored